# Loop 4 Analysis: CV-LB Gap Investigation

The evaluator correctly identified that we need to validate exp_003 (Combined Spange + DRFP) on LB.

## Key Questions:
1. Why is there a 9x gap between CV and LB?
2. What can we learn from the submission history?
3. What should we try next if LB doesn't improve?

In [1]:
import pandas as pd
import numpy as np

# Analyze submission history
submissions = [
    {'exp': 'exp_000', 'model': 'MLP Spange+Arrhenius', 'cv': 0.0111, 'lb': 0.0982},
    {'exp': 'exp_001', 'model': 'LightGBM', 'cv': 0.0123, 'lb': 0.1065},
]

df = pd.DataFrame(submissions)
df['cv_lb_ratio'] = df['lb'] / df['cv']
df['cv_lb_gap'] = df['lb'] - df['cv']
print('Submission History Analysis:')
print(df.to_string(index=False))
print(f'\nAverage CV-LB ratio: {df["cv_lb_ratio"].mean():.2f}x')
print(f'Average CV-LB gap: {df["cv_lb_gap"].mean():.4f}')

Submission History Analysis:
    exp                model     cv     lb  cv_lb_ratio  cv_lb_gap
exp_000 MLP Spange+Arrhenius 0.0111 0.0982     8.846847     0.0871
exp_001             LightGBM 0.0123 0.1065     8.658537     0.0942

Average CV-LB ratio: 8.75x
Average CV-LB gap: 0.0906


In [2]:
# If the CV-LB ratio holds, what would exp_003 LB be?
exp003_cv = 0.010501
avg_ratio = df['cv_lb_ratio'].mean()
predicted_lb = exp003_cv * avg_ratio

print(f'exp_003 CV: {exp003_cv:.6f}')
print(f'Predicted LB (using avg ratio {avg_ratio:.2f}x): {predicted_lb:.4f}')
print(f'\nTarget: 0.0333')
print(f'Current best LB: 0.0982')
print(f'\nIf CV-LB ratio holds:')
print(f'  - exp_003 would have LB ~{predicted_lb:.4f}')
print(f'  - This is still ~{predicted_lb/0.0333:.1f}x worse than target')

exp_003 CV: 0.010501
Predicted LB (using avg ratio 8.75x): 0.0919

Target: 0.0333
Current best LB: 0.0982

If CV-LB ratio holds:
  - exp_003 would have LB ~0.0919
  - This is still ~2.8x worse than target


In [3]:
# What CV would we need to beat the target?
target = 0.0333
required_cv = target / avg_ratio
print(f'To beat target {target}:')
print(f'  - With avg ratio {avg_ratio:.2f}x, we need CV < {required_cv:.6f}')
print(f'  - Current best CV: {exp003_cv:.6f}')
print(f'  - Gap to close: {exp003_cv - required_cv:.6f}')
print(f'\nThis suggests we need ~{exp003_cv/required_cv:.1f}x improvement in CV')

To beat target 0.0333:
  - With avg ratio 8.75x, we need CV < 0.003805
  - Current best CV: 0.010501
  - Gap to close: 0.006696

This suggests we need ~2.8x improvement in CV


In [4]:
# Analyze the competition structure
print('Competition Evaluation Structure:')
print('='*50)
print('The competition runs the notebook on Kaggle servers.')
print('This means:')
print('1. Different random seeds/environment')
print('2. The CV score IS the LB score (no separate test set)')
print('3. The 9x gap is due to model variance, not data shift')
print()
print('Key insight: The LB score IS a CV score, just with different randomness.')
print('Our local CV is optimistic because we use fixed seeds.')
print()
print('Implications:')
print('- Reducing model variance is critical')
print('- More bagging/ensembling should help')
print('- Deterministic models should have lower gap (but LightGBM was worse)')

Competition Evaluation Structure:
The competition runs the notebook on Kaggle servers.
This means:
1. Different random seeds/environment
2. The CV score IS the LB score (no separate test set)
3. The 9x gap is due to model variance, not data shift

Key insight: The LB score IS a CV score, just with different randomness.
Our local CV is optimistic because we use fixed seeds.

Implications:
- Reducing model variance is critical
- More bagging/ensembling should help
- Deterministic models should have lower gap (but LightGBM was worse)


In [5]:
# What approaches haven't been tried?
print('Approaches NOT yet tried:')
print('='*50)
print('1. More aggressive ensembling (10+ models instead of 5)')
print('2. Different architectures (deeper/wider networks)')
print('3. Different loss functions (MSE vs Huber)')
print('4. Learning rate schedules (cosine annealing)')
print('5. Task-specific models (different features for single vs mixture)')
print('6. Fragprints features (2133 features available)')
print('7. ACS PCA descriptors (5 features available)')
print('8. Gaussian Process models (uncertainty quantification)')
print()
print('Evaluator recommendation: Submit exp_003 first to validate CV improvement')

Approaches NOT yet tried:
1. More aggressive ensembling (10+ models instead of 5)
2. Different architectures (deeper/wider networks)
3. Different loss functions (MSE vs Huber)
4. Learning rate schedules (cosine annealing)
5. Task-specific models (different features for single vs mixture)
6. Fragprints features (2133 features available)
7. ACS PCA descriptors (5 features available)
8. Gaussian Process models (uncertainty quantification)

Evaluator recommendation: Submit exp_003 first to validate CV improvement
